In [17]:
print("salut c'est moi")

salut c'est moi


In [16]:
import os

print(os.environ.keys())

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

KeysView(environ({'SHELL': '/bin/bash', 'NV_LIBCUBLAS_VERSION': '12.4.5.8-1', 'NVIDIA_VISIBLE_DEVICES': 'void', 'KUBERNETES_SERVICE_PORT_HTTPS': '443', 'NV_NVML_DEV_VERSION': '12.4.127-1', 'NV_CUDNN_PACKAGE_NAME': 'libcudnn9-cuda-12', 'KUBERNETES_SERVICE_PORT': '443', 'AWS_S3_ENDPOINT': 'minio.lab.sspcloud.fr', 'WORKSPACE_DIR': '/home/onyxia/work', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.21.5-1+cuda12.4', 'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.21.5-1', 'HOSTNAME': 'jupyter-python-gpu-901276-0', 'PYTHON_VERSION': '3.12.9', 'GIT_USER_MAIL': 'adrien.quentin@insee.fr', 'DARK_MODE': 'true', 'NVIDIA_REQUIRE_CUDA': 'cuda>=12.4 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,driver<471 b

In [18]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

RuntimeError: Unsloth: You already added LoRA adapters to your model!

In [19]:
from unsloth import FastModel
import torch
max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


In [20]:
from datasets import load_dataset

# Load CSV into Hugging Face dataset
dataset = load_dataset("csv", data_files="fr-train-dataset2.csv")

# Access the split (train by default)
train_dataset = dataset["train"]

# Preview
print(train_dataset[0])
train_dataset

Generating train split: 2695 examples [00:00, 31003.13 examples/s]

{'Address': 'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France', 'Building_Number': 147, 'City': 'Paris', 'Recipient': 'Supermarché Carrefour', 'Street_Name_old': 'Rue de Rivoli', 'Street_Name': 'de Rivoli', 'type_voie': 'Rue', 'Zip_Code': 75001, 'Country': 'France', 'repetition': None}


Dataset({
    features: ['Address', 'Building_Number', 'City', 'Recipient', 'Street_Name_old', 'Street_Name', 'type_voie', 'Zip_Code', 'Country', 'repetition'],
    num_rows: 2695
})

In [21]:
train_dataset[0]["Address"]

'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France'

In [22]:
# Define columns to exclude from tgt
exclude_cols = ["Address", "Street_Name_old"]  # 👈 Add any column to ignore here

# Determine tgt columns
tgt_cols = [col for col in train_dataset.column_names if col not in exclude_cols]
src_col = "Address"


# Transform
def transform(example):
    return {
        "question": example[src_col],
        "answer": {col: example[col] for col in tgt_cols}
    }

dataset_refined = train_dataset.map(transform)

# Preview
print(dataset_refined[0])

Map: 100%|██████████| 2695/2695 [00:01<00:00, 2261.70 examples/s]

{'Address': 'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France', 'Building_Number': 147, 'City': 'Paris', 'Recipient': 'Supermarché Carrefour', 'Street_Name_old': 'Rue de Rivoli', 'Street_Name': 'de Rivoli', 'type_voie': 'Rue', 'Zip_Code': 75001, 'Country': 'France', 'repetition': None, 'question': 'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France', 'answer': {'Building_Number': 147, 'City': 'Paris', 'Country': 'France', 'Recipient': 'Supermarché Carrefour', 'Street_Name': 'de Rivoli', 'Zip_Code': 75001, 'repetition': None, 'type_voie': 'Rue'}}


In [23]:
dataset_refined[0]["answer"]

{'Building_Number': 147,
 'City': 'Paris',
 'Country': 'France',
 'Recipient': 'Supermarché Carrefour',
 'Street_Name': 'de Rivoli',
 'Zip_Code': 75001,
 'repetition': None,
 'type_voie': 'Rue'}

In [24]:
dataset_refined[0]["question"]

'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France'